In [1]:
from __future__ import print_function

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import astropy.units as units
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astroquery.irsa_dust import IrsaDust

import healpy as hp
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from dustmaps.sfd import SFDQuery
from dustmaps.planck import PlanckQuery
from dustmaps.bayestar import BayestarQuery




Configuration file not found:

    /Users/supernova/.dustmapsrc

To create a new configuration file in the default location, run the following python code:

    from dustmaps.config import config
    config.reset()

Note that this will delete your configuration! For example, if you have specified a data directory, then dustmaps will forget about its location.


In [2]:
import os

In [4]:

ebv_map = hp.read_map('../externaldata/ebv_lhd.hpx.fits', verbose=False)
nside = hp.get_nside(ebv_map)
npix = hp.nside2npix(nside)
ordering = 'ring'



In [5]:
name='M101'
coord = SkyCoord.from_name(name, frame = 'icrs')
s_gal = coord.galactic
glon = s_gal.l.value
glat = s_gal.b.value
# get pixel numbers
galpix = hp.ang2pix(nside, glon, glat, lonlat=True)
print(galpix)
# get reddening for these pixels
ebv_los = ebv_map[galpix]
print(ebv_los)


854865
0.009145982


In [6]:
imagename=name+'_4comparison.png'
ra0 = coord.ra
dec0 = coord.dec


In [7]:
ra = np.arange(ra0.degree - 2., ra0.degree + 2., 0.05)
dec = np.arange(dec0.degree - 2., dec0.degree + 2., 0.05)

ragrid, decgrid = np.meshgrid(ra, dec)
coords = SkyCoord(ragrid*units.deg, decgrid*units.deg, frame='icrs')

sfd = SFDQuery()
# this conversion puts Av on the Schlafly system
Av_sfd = 2.742 * sfd(coords)

planck = PlanckQuery()
Av_planck = 3.1 * planck(coords)


The SFD'98 dust map is not in the data directory:

    /usr/local/lib/python3.9/site-packages/dustmaps/data

To change the data directory, call:

    from dustmaps.config import config
    config['data_dir'] = '/path/to/data/directory'

To download the SFD'98 map to the data directory, call:

    import dustmaps.sfd
    dustmaps.sfd.fetch()



FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.9/site-packages/dustmaps/data/sfd/SFD_dust_4096_ngp.fits'

In [ ]:

coords = SkyCoord(ragrid*units.deg, decgrid*units.deg,
                  distance=1000000.*units.kpc, frame='galactic')
bayestar = BayestarQuery(max_samples=1)
Av_bayestar = 2.742 * bayestar(coords)
print('done with Bayestar')

Av_HI=0*Av_sfd

In [ ]:
maxboth=max(np.amax(Av_planck),np.amax(Av_sfd))
print(maxboth)

In [ ]:
print(np.shape(Av_HI))

In [ ]:
nside = hp.get_nside(ebv_map)

npix = hp.nside2npix(nside)
ordering = 'ring'
s_gal = coords.galactic

glon = s_gal.l.value
glat = s_gal.b.value
# get pixel numbers
pix = hp.ang2pix(nside, glon, glat, lonlat=True)
#print(pix)
# get reddening for these pixels
ebv_los = ebv_map[pix]
for i in range(len(dec)-1):
    for j in range(len(ra)-1):
        stepcoord = SkyCoord(ra[i]*units.deg, dec[j]*units.deg, frame='icrs')
        print(stepcoord)
        stepgal = stepcoord.galactic
        steplon = stepgal.l.value
        steplat = stepgal.b.value
        # get pixel numbers
        steppix = hp.ang2pix(nside, steplon, steplat, lonlat=True)
        print(steppix)
        ebv=ebv_map[steppix]
        Av_HI[j,i]=ebv*2.742
    #print(ebv)
#AV_HI=2.742*ebv_los
#print('done with AV_HI')

In [ ]:
print(Av_HI)

In [ ]:
#fig = plt.figure(figsize=(8,8), dpi=150)

#for k,(Av,title) in enumerate([(Av_sfd, 'SFD'),
#                               (Av_planck, 'Planck'),
#                         #      (AV_HI, 'HI'),
#                               (Av_bayestar, 'Bayestar')]):
#    ax = fig.add_subplot(2,2,k+1)
fig = plt.figure(figsize=(8,8), dpi=150)

for k,(Av,title) in enumerate([(Av_sfd, 'SFD'),
                               (Av_planck, 'Planck'),
                               (Av_HI, 'HI'),
                               (Av_bayestar, 'Bayestar')]):
    ax = fig.add_subplot(2,2,k+1)
    ax.imshow(
        np.sqrt(Av)[::,::-1],
        vmin=0.,
        vmax=0.9,
        origin='lower',
        interpolation='nearest',
        cmap='binary',
        aspect='equal'
    )
    ax.axis('off')
    ax.set_title(title)

fig.subplots_adjust(wspace=0., hspace=0.)
plt.savefig(imagename, dpi=150)
